In [1]:
# Loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


In [2]:
file_path = "anonymized_challenge_dataset.csv"
dataset = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
dataset.head()


,ID,interview_date,country,NPS,comment,translated_comment
0,1,2022-05-11,Germany,0,Die Vertragsänderung meiner Mutter wurde nicht...,My mother&#39;s contract change did not go thr...
1,2,2022-07-19,Poland,0,ebok nie dziala - brak wgladu do Faktur,ebok does not work - no access to invoices
2,3,2022-12-14,Italy,8,NON SAPREI. PERCHE' MI SONO TROVATO BENE SEMPR...,I WOULD NOT KNOW. BECAUSE I HAVE ALWAYS FOUND ...
3,4,2022-08-04,Germany,10,Service auch telefonisch immer erreichbar und ...,Service is always available by phone and you g...
4,5,2022-08-15,Poland,9,"Fachowosc,kompetencje i kultura pracownika","Professionalism, competence and employee culture"


In [3]:
# Data Cleaning
dataset = dataset.drop("comment", axis = 1)
print(dataset.info())

# Check for missing values
missing_values = dataset.isnull().sum()

# Initial analysis of NPS values (checking for outliers or inappropriate values)
dataset['NPS'] = pd.to_numeric(dataset['NPS'], errors='coerce')
nps_summary = dataset['NPS'].describe()

# Convert 'interview_date' to datetime
dataset['interview_date'] = pd.to_datetime(dataset['interview_date'], errors='coerce')

# Check data types
data_types = dataset.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503952 entries, 0 to 503951
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ID                  503952 non-null  int64 
 1   interview_date      502936 non-null  object
 2   country             502566 non-null  object
 3   NPS                 500699 non-null  object
 4   translated_comment  497321 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.2+ MB
None


In [4]:
missing_values

ID                       0
interview_date        1016
country               1386
NPS                   3253
translated_comment    6631
dtype: int64

In [5]:
data_types

ID                             int64
interview_date        datetime64[ns]
country                       object
NPS                          float64
translated_comment            object
dtype: object

In [6]:
dataset['NPS'].describe()

count    500525.000000
mean          7.711479
std           2.880806
min           0.000000
25%           7.000000
50%           9.000000
75%          10.000000
max          99.000000
Name: NPS, dtype: float64

In [7]:
len(dataset[dataset['NPS'] > 10])

15

In [8]:
# Investigating the outlier in NPS
dataset = dataset[dataset['NPS'] < 11]

# Recheck NPS summary after handling outliers
nps_summary_cleaned = dataset['NPS'].describe()

dataset = dataset[dataset['translated_comment'].notnull()]
dataset.dropna(subset=['interview_date'], inplace=True)


# Convert results to a dictionary for easier interpretation
cleaning_results = {
    "missing_values": dataset.isnull().sum().to_dict(),
    "nps_summary_cleaned": nps_summary_cleaned.to_dict()
}

cleaning_results


{'missing_values': {'ID': 0,
  'interview_date': 0,
  'country': 0,
  'NPS': 0,
  'translated_comment': 0},
 'nps_summary_cleaned': {'count': 500510.0,
  'mean': 7.708743082056302,
  'std': 2.837169444802836,
  'min': 0.0,
  '25%': 7.0,
  '50%': 9.0,
  '75%': 10.0,
  'max': 10.0}}

In [9]:
# Remove junk and map Czech to Czech Republic
keep_countries = ['Germany', 'Poland', 'Italy', 'Romania', 'Sweden', 'Netherlands', 'Czech',
                          'Czech Republic', 'United Kingdom', 'Hungary']
df = dataset[dataset['country'].isin(keep_countries)]
df['country'].value_counts()

Netherlands       153148
Czech             130030
Germany            48484
Romania            41674
Poland             34981
Italy              32409
Sweden             30512
Hungary            24994
Czech Republic       667
United Kingdom       397
Name: country, dtype: int64

In [10]:
country_mapping = {'Czech Republic': 'Czech'}
df['country'] = df['country'].replace(country_mapping)

In [11]:
df['country'].value_counts()

Netherlands       153148
Czech             130697
Germany            48484
Romania            41674
Poland             34981
Italy              32409
Sweden             30512
Hungary            24994
United Kingdom       397
Name: country, dtype: int64

In [12]:
# Categorize customers into different types
def customerType(dataset):
    if dataset["NPS"] >= 9:
        return "promoters"
    elif (dataset["NPS"] == 7 or dataset["NPS"] == 8):
        return "passives"
    elif dataset["NPS"] <= 6:
        return "detractors" 
    
df["customer_type"] = dataset.apply(customerType, axis = 1)

In [13]:
df['comment_length'] = df['translated_comment'].str.len()

In [14]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import re

# Downloading necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

sw_nltk = stopwords.words('english')
sw_nltk.remove('not')
sw_nltk.remove('no')
sw_nltk.remove('nor')
sw_nltk.remove("don't")
# sw_nltk.remove("aren't")
# sw_nltk.remove("couldn't")
# sw_nltk.remove("didn't")
stop_words = [word for word in sw_nltk if word not in sw_nltk[-36:]]
# sw_nltk.remove([sw_nltk[-36:]])

# Initializing the lemmatizer
lemmatizer = WordNetLemmatizer()



[nltk_data] Downloading package punkt to /Users/adam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/adam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/adam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import string
def preprocess_text(text):
    # Convert to lower case
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    words = nltk.word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # Joining back into a string
    text = ' '.join(words)
    return text

# df['comment_preprocessed'] = df['comment'].apply(preprocess_text)
df["translated_comment"] = df["translated_comment"].str.replace("&#39;","\'")
df["translated_comment"] = df["translated_comment"].str.replace("nee", "no", regex = False)
df["translated_comment"] = df["translated_comment"].str.replace("Nee", "No", regex = False)
df["translated_comment"] = df["translated_comment"].str.replace("essent", "eon", regex = False)
df["translated_comment"] = df["translated_comment"].str.replace("Essent", "Eon", regex = False)
df["translated_comment"] = df["translated_comment"].str.replace("quot", "", regex = False)

df["translated_comment"] = df["translated_comment"].replace(r'\d+', '', regex=True)

df['translated_comment_preprocessed'] = df['translated_comment'].apply(preprocess_text)

# Define a function to lemmatize text
def lemmatize_text(text):
    # Tokenize the sentence
    tokens = word_tokenize(text)
    # Lemmatize each word in the sentence
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join the lemmatized tokens back into a sentence
    lemmatized_sentence = ' '.join(lemmatized_tokens)
    return lemmatized_sentence

# Apply the function to the DataFrame
df['translated_comment_preprocessed'] = df['translated_comment_preprocessed'].apply(lemmatize_text)

In [16]:
df[['NPS', 'translated_comment', 'translated_comment_preprocessed']].head(20)

,NPS,translated_comment,translated_comment_preprocessed
0,0.0,My mother's contract change did not go through...,mother contract change not go expected bank de...
1,0.0,ebok does not work - no access to invoices,ebok not work no access invoice
2,8.0,I WOULD NOT KNOW. BECAUSE I HAVE ALWAYS FOUND ...,would not know always found well eon output pr...
3,10.0,Service is always available by phone and you g...,service always available phone get help
4,9.0,"Professionalism, competence and employee culture",professionalism competence employee culture
5,10.0,easy to read and simple IT system,easy read simple system
6,10.0,Advice 'at the window' in the Warsaw center of...,advice window warsaw center mokotow efficient ...
7,10.0,"very quickly the problem was solved, the count...",quickly problem solved counter not register ca...
8,1.0,The contract I got,contract got
9,8.0,Better price.,better price


In [17]:
dataset = df.copy()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497296 entries, 0 to 503951
Data columns (total 8 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   ID                               497296 non-null  int64         
 1   interview_date                   497296 non-null  datetime64[ns]
 2   country                          497296 non-null  object        
 3   NPS                              497296 non-null  float64       
 4   translated_comment               497296 non-null  object        
 5   customer_type                    497296 non-null  object        
 6   comment_length                   497296 non-null  int64         
 7   translated_comment_preprocessed  497296 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 34.1+ MB


In [18]:
def is_numeric(text):
    """
    Check if the given text is entirely numeric.
    """
    return text.isdigit() if isinstance(text, str) else False

# Filter out rows where the comments are entirely numeric
dataset = dataset[~dataset['translated_comment_preprocessed'].apply(is_numeric)]

In [19]:
import re

def remove_emojis(text):
    """
    Remove emojis from the given text.
    """
    # Emoji patterns
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Apply the function to the relevant columns
dataset['translated_comment_preprocessed'] = dataset['translated_comment_preprocessed'].apply(remove_emojis)
dataset['translated_comment'] = dataset['translated_comment'].apply(remove_emojis)

In [20]:
dataset = dataset[dataset['comment_length'] > 1]

dataset['comment_length_2'] = dataset['translated_comment_preprocessed'].str.len()
dataset = dataset[dataset['comment_length_2'] > 0]

dataset = dataset.dropna(subset=["translated_comment_preprocessed"])
dataset.drop('comment_length_2', axis=1, inplace=True)

In [21]:
dataset.to_csv('cleaned_dataset.csv', index=False)

In [22]:
# Save 10 percent of dataset for faster computing in some algorithms
df_percent = dataset.sample(frac=0.1)
df_percent.to_csv('cleaned_dataset_10percent.csv', index=False)